In [1]:
pip list

Package                      Version             
---------------------------- --------------------
absl-py                      0.15.0              
alembic                      1.7.5               
apturl                       0.5.2               
argon2-cffi                  21.3.0              
argon2-cffi-bindings         21.2.0              
asttokens                    2.0.5               
astunparse                   1.6.3               
atomicwrites                 1.1.5               
attrs                        19.3.0              
autopage                     0.4.0               
backcall                     0.2.0               
bcrypt                       3.1.7               
beautifulsoup4               4.8.2               
black                        21.12b0             
bleach                       4.1.0               
blinker                      1.4                 
Brlapi                       0.7.0               
cachetools                   4.2.4               


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow as tf
from keras.models import Model
import copy



In [3]:
#disable the GPU

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

2023-09-29 17:20:46.193498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-29 17:20:47.765774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-29 17:20:47.767756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
# specifying constant parameters

T_0 = 300.0
V = 1.0
k_0 = 8.46*(np.power(10,6))
C_p = 0.231
rho_L = 1000.0
Q_s = 0.0
T_s = 402.0
F = 5.0
E = 5.0*(np.power(10,4))
delta_H = -1.15*(np.power(10,4))
R = 8.314
C_A0s = 4.0
C_As = 1.95
t_final = 0.01
t_step = 1e-4
P = np.array([[1060.0, 22.0], [22.0, 0.52]])


In [39]:
# generating inputs and initial states for CSTR, all expressed in deviation form

u1_list = np.linspace(-3.5, 3.5, 2, endpoint=True)
u2_list = np.linspace(-5e5 * 1.0, 5e5 * 1.0, 2, endpoint=True)
T_initial = np.linspace(300.0, 600.0, 400, endpoint=True) - T_s
CA_initial = np.linspace(0.0, 6.0, 400, endpoint=True) - C_As


In [40]:
# sieve out initial states that lie outside of stability region

T_start = list()
CA_start = list()

for T in T_initial:
    for CA in CA_initial:
        x = np.array([CA, T])
        if x @ P @ x < 372:
            CA_start.append(CA)
            T_start.append(T)
print("number of initial conditions: {}".format(len(CA_start)))

# convert to np.arrays
CA_start = np.array([CA_start])
T_start = np.array([T_start])
x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)  # every row is a pair of initial states within stability region
print("shape of x_deviation is {}".format(x_deviation.shape))

number of initial conditions: 12603
shape of x_deviation is (12603, 2)


In [41]:
def CSTR_simulation_derivative( C_A_initial, T_initial, C_A0, Q):
    
    C_A = C_A_initial + C_As
    T = T_initial + T_s
    
    dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
    dTdt = F / V * (T_0 - T) -  delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)
    
    return dCAdt, dTdt

In [42]:
# get X and y data for training and testing

dCAdt_output = list()
dTdt_output = list()
CA_input = list()
T_input = list()
CA0_input = list()
Q_input = list()

for u1 in u1_list:
    C_A0 = u1 + C_A0s
    
    for u2 in u2_list:
        Q = u2 + Q_s
        
        for C_A_initial, T_initial in x_deviation:
            CA0_input.append(u1)
            Q_input.append(u2)
            CA_input.append(C_A_initial)
            T_input.append(T_initial)
            
            dCAdt, dTdt = CSTR_simulation_derivative( C_A_initial, T_initial, C_A0, Q)
            dCAdt_output.append([dCAdt])
            dTdt_output.append([dTdt])


In [43]:
# collate input for RNN

CA0_input = np.array(CA0_input)
CA0_input = CA0_input.reshape(-1,1)
print("CA0_input shape is {}".format(CA0_input.shape))

Q_input = np.array(Q_input)
Q_input = Q_input.reshape(-1,1)
print("q_input shape is {}".format(Q_input.shape))

CA_input = np.array(CA_input)
CA_input = CA_input.reshape(-1,1)
print("CA_input shape is {}".format(CA_input.shape))

T_input = np.array(T_input)
T_input = T_input.reshape(-1,1)
print("T_input shape is {}".format(T_input.shape))

NN_input = np.concatenate(( CA_input, T_input, CA0_input, Q_input), axis=1)
print("NN_input shape is {}".format(NN_input.shape))



CA0_input shape is (50412, 1)
q_input shape is (50412, 1)
CA_input shape is (50412, 1)
T_input shape is (50412, 1)
NN_input shape is (50412, 4)


In [44]:

print(NN_input[0, 0])
print(NN_input[0, 1])

1.5537593984962406
-76.43609022556393


In [45]:
# collate output for RNN

dCAdt_output = np.array(dCAdt_output)
dCAdt_output = dCAdt_output.reshape(-1,1)

dTdt_output = np.array(dTdt_output)
dTdt_output = dTdt_output.reshape(-1, 1)

NN_output = np.concatenate((dCAdt_output, dTdt_output), axis=1)
print("RNN_output shape is {}".format(NN_output.shape))  # output shape: number of samples x timestep x variables

RNN_output shape is (50412, 2)


In [12]:
import copy
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(NN_input, NN_output, test_size=0.3, random_state=123)

X_train_copy = tf.constant(copy.deepcopy(X_train), dtype = tf.float32)
X_test_copy = tf.constant(copy.deepcopy(X_test), dtype = tf.float32)
X_all_copy = tf.constant(copy.deepcopy(NN_input), dtype = tf.float32)


# define scalers for both X and y base on training data only
scaler_X = preprocessing.StandardScaler(with_mean = False).fit(X_train)
scaler_y = preprocessing.StandardScaler(with_mean = False).fit(y_train)

print(scaler_X.mean_)
print(scaler_y.mean_)
print(scaler_X.scale_)
print(scaler_y.scale_)

# These are basically tf (tensorflow) matrices to convert the x and u from the original form into normalized form and convert y from normalized
# form to original form. So D_x x is the normalized version of x, and D_u u is the normalized version of u, and D_y (output) is the original form of y.

D_x = tf.constant([[1/scaler_X.scale_[0],0],[0,1/scaler_X.scale_[1]]],dtype = tf.float32)
D_u = tf.constant([[1/scaler_X.scale_[2],0],[0,1/scaler_X.scale_[3]]],dtype = tf.float32)
D_y = tf.constant([[scaler_y.scale_[0],0],[0,scaler_y.scale_[1]]], dtype = tf.float32)

print(D_u)
print(D_y)

X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.transform(y_train)
y_test = scaler_y.transform(y_test)

[ 9.87558770e-03 -3.23761302e-01  8.00000000e-03  1.17142857e+03]
[  1.33820414 -60.11610829]
[8.48111755e-01 3.83758131e+01 2.47641596e+00 3.53440315e+05]
[  13.73299569 1548.55957516]
tf.Tensor(
[[4.0380937e-01 0.0000000e+00]
 [0.0000000e+00 2.8293321e-06]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[  13.732996    0.      ]
 [   0.       1548.5596  ]], shape=(2, 2), dtype=float32)


2023-09-29 17:21:17.245894: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# checking X_train
print(X_train[0, 0])
print(X_train[0, 1])

1.820126741718589
-1.8662862721343012


In [14]:

# This function outputs the value of sontag controller  u interms of a (LfV) and b (LgV). I used a and b this way throughout the rest of the code. 
# Remember that in the sontag law we only use 1 controlled
# variable, which is the heat input Q. 



def phi(a,b):
    factor = - (a + np.sqrt(a**2 + b**4)) / (b**2)
    psi  =  factor * b
    if psi > 500000:
        psi = 500000
    if psi < -500000:
        psi = -500000
    #print(a,b,psi)
    return psi 

def phi_2(a,b):  ###get CA0
    factor = - (a + np.sqrt(a**2 + b**4)) / (b**2)
    psi  =  factor * b
    if psi > 3.5:
        psi = 3.5
    if psi < -3.5:
        psi = -3.5
    #print(a,b,psi)
    return psi 

In [15]:
def CSTR_simulation_derivative2( C_A_initial, T_initial, C_A0, Q):
    
    C_A = C_A_initial + C_As
    T = T_initial + T_s
    
    dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
    dTdt = F / V * (T_0 - T) -  delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)
    
    return tf.constant(np.array([dCAdt, dTdt]), shape = (2,1))


### this function checks if V is leq 5, and if so, just return a very large negative value. The reason is because this very large negative value 
### will output 0 when passed into the ReLU function.

def modified_dVdt(C_A, T, dVdt):
    x = np.array([C_A,T])
    V = x @ P @ x
    if V <= 12:
        return -1e5
    else:
        return dVdt

### this is f for the FP model in Eq. 1
def f_fp(C_A, T):
    C_A = C_A + C_As
    T = T + T_s

    f1 = F / V * (- C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
    f2 = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 
    f1s = F / V * (- C_As) - k_0 * np.exp(-E / (R * T_s)) * C_As** 2
    f2s = F / V * (T_0 - T_s) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T_s)) * C_As**2
    
    return tf.constant(np.array([f1-f1s, f2-f2s]), shape = (2,1))

### this is g for the FP model in Eq. 1
def g_fp():
    return tf.constant(np.array([[F / V , 0],[0, 1/ (rho_L * C_p * V)]]) , shape = (2,2), dtype = tf.float32)

### this is dVdt using Sontag controller without taking into account states that are below \rho_s
def dVsontag_fp(curr_states):
    input_tensor = tf.reshape(curr_states, (curr_states.shape[0] , 1 , 2))
    two_times_x_times_P = tf.stack([ 2 * elem @ tf.convert_to_tensor(P, dtype = tf.float32) for elem in tf.unstack(input_tensor)]) 
    f = tf.map_fn( lambda x : f_fp(x[0,0],x[0,1]) , input_tensor)
    LfV = tf.matmul(two_times_x_times_P, f)
    a = LfV
    g = tf.stack([g_fp() for elem in tf.unstack(input_tensor)])
    dVdx_times_g = tf.matmul(two_times_x_times_P , g)
    b =  tf.gather(dVdx_times_g, [1], axis = 2)
    a_b = tf.concat([a,b],axis = 2)
    ab = tf.reshape(a_b, (curr_states.shape[0], 2))
    sontags = tf.reshape(tf.map_fn(lambda elem : phi(elem[0],elem[1]) , ab ), (curr_states.shape[0],1,1))
    LgVu = tf.matmul(b,sontags)
    dVdt = tf.reshape(a + LgVu,(curr_states.shape[0],1))
    return dVdt 

### this is dVdt using Sontag controller taking into account states that are below \rho_s
def dVsontag_fp_modified(curr_states):
    dVdt = dVsontag_fp(curr_states)
    new_input = tf.concat([input_tensor, dVdt], axis = 2)
    inputs2 = tf.reshape(new_input , (curr_states.shape[0],3))
    return tf.map_fn(lambda elem : modified_dVdt(elem[0],elem[1],elem[2]) , inputs2)


### this is Sontag controller without taking into account states that are below \rho_s
def dVsontag_fp_sontag(curr_states):
    input_tensor = tf.reshape(curr_states, (curr_states.shape[0] , 1 , 2))
    two_times_x_times_P = tf.stack([ 2 * elem @ tf.convert_to_tensor(P, dtype = tf.float32) for elem in tf.unstack(input_tensor)]) 
    f = tf.map_fn( lambda x : f_fp(x[0,0],x[0,1]) , input_tensor)
    LfV = tf.matmul(two_times_x_times_P, f)
    a = LfV
    g = tf.stack([g_fp() for elem in tf.unstack(input_tensor)])
    dVdx_times_g = tf.matmul(two_times_x_times_P , g)
    b =  tf.gather(dVdx_times_g, [1], axis = 2)
    a_b = tf.concat([a,b],axis = 2)
    ab = tf.reshape(a_b, (curr_states.shape[0], 2))
    sontags = tf.reshape(tf.map_fn(lambda elem : phi(elem[0],elem[1]) , ab ), (curr_states.shape[0],1,1))
    
    return sontags


### this is Sontag controller taking into account states that are below \rho_s
def dVsontag_fp_modified_sontag(curr_states):
    input_tensor = tf.reshape(curr_states, (curr_states.shape[0] , 1 , 2))
    dVdt = dVsontag_fp(curr_states)
    new_input = tf.concat([input_tensor, dVdt], axis = 2)
    inputs2 = tf.reshape(new_input , (curr_states.shape[0],3))
    return inputs2



class ControlAffineNN(tf.keras.Model):

    def __init__(self, state_dim, control_dim, f_layers_list, g_layers_list):
        super(ControlAffineNN, self).__init__()
        self.control_dim = control_dim
        self.state_dim = state_dim
        self.f_layers_list = f_layers_list
        self.g_layers_list = g_layers_list

        self.f_layers = []
        for i in range(len(f_layers_list)):
            self.f_layers.append(Dense(f_layers_list[i], activation = 'relu'))
        self.f_layers.append(Dense(state_dim, activation = 'linear'))

        self.g_layers = []
        for i in range(len(g_layers_list)):
            self.g_layers.append(Dense(g_layers_list[i], activation = 'relu'))
        self.g_layers.append(Dense(state_dim * control_dim, activation = 'linear'))
    
    def call(self, inputs):
        print(inputs.shape[0])
        state = inputs[:,0:self.state_dim]
        control = tf.reshape(inputs[:,self.state_dim::], (inputs.shape[0], self.control_dim , 1 ))

        f = tf.identity(state)
        for layer in self.f_layers:
            f = layer(f)

        g = tf.identity(state)
        for layer in self.g_layers:
            g = layer(g)      
        g = tf.reshape( g, (inputs.shape[0], self.state_dim , self.control_dim ) )


        gu = tf.reshape(tf.matmul(g,control) , (inputs.shape[0], self.state_dim))
    
        return f + gu
    
    def get_config(self):
        config = super(ControlAffineNN, self).get_config()
        config['state_dim'] = self.state_dim
        config['control_dim'] = self.control_dim
        config['f_layers_list'] = self.f_layers_list
        config['g_layers_list'] = self.g_layers_list
        return config

### this is f for the NN model
    def f_nn(self,curr_states):
        f = tf.identity(curr_states)
        for layer in self.f_layers:
            f = layer(f)
        return f

### this is g for the NN model

    def g_nn(self,curr_states):
        g = tf.identity(curr_states)
        for layer in self.g_layers:
            g = layer(g)      
        g = tf.reshape( g, (curr_states.shape[0], self.state_dim , self.control_dim ) )
        return g

### this is F_nn for the NN model

    def dxdt(self,curr_states_unnormalized):
        curr_states = tf.constant(scaler_X.transform(curr_states_unnormalized),dtype = tf.float32)
        dxdt_normalized = self.call(curr_states)
        return scaler_y.inverse_transform(dxdt_normalized)


### this is dVdt for the NN model [\nabla V F_nn (x, sontag(x))]
### this is where the D_x, D_u and D_y are used

    def dVsontag(self,curr_states):
        input_tensor = tf.reshape(curr_states, (curr_states.shape[0] , 1 , self.state_dim))
        two_times_x_times_P = tf.stack([ 2 * elem @ tf.convert_to_tensor(P, dtype = tf.float32) for elem in tf.unstack(input_tensor)])

        tildecurr_states = tf.stack([ elem @ D_x for elem in tf.unstack(input_tensor)])

        tildef_nn = tf.reshape(self.f_nn(tildecurr_states), (curr_states.shape[0] ,self.state_dim , 1))

        fnn = tf.stack([ D_y @ elem  for elem in tf.unstack(tildef_nn)])
    
        f = tf.reshape(fnn, (curr_states.shape[0] , self.state_dim, 1))
        a = tf.matmul(two_times_x_times_P, f) ### LfV

        tildeg = self.g_nn(tildecurr_states)
        g = tf.stack([ D_y @ elem @ D_u for elem in tf.unstack(tildeg)])
        dVdx_times_g = tf.matmul(two_times_x_times_P , g)

        b =  tf.gather(dVdx_times_g, [1], axis = 2)
        a_b = tf.concat([a,b],axis = 2)
        ab = tf.reshape(a_b, (curr_states.shape[0], 2))
        sontags = tf.reshape(tf.map_fn(lambda elem : phi(elem[0],elem[1]) , ab ), (curr_states.shape[0],1,1))
        LgVu = tf.matmul(b,sontags)
        return tf.reshape(a + LgVu,(curr_states.shape[0],1))
    
    ###get the value of sontag law
    def dVsontag_value(self,curr_states):
        input_tensor = tf.reshape(curr_states, (curr_states.shape[0] , 1 , self.state_dim))
        two_times_x_times_P = tf.stack([ 2 * elem @ tf.convert_to_tensor(P, dtype = tf.float32) for elem in tf.unstack(input_tensor)])

        tildecurr_states = tf.stack([ elem @ D_x for elem in tf.unstack(input_tensor)])

        tildef_nn = tf.reshape(self.f_nn(tildecurr_states), (curr_states.shape[0] ,self.state_dim , 1))

        fnn = tf.stack([ D_y @ elem  for elem in tf.unstack(tildef_nn)])
    
        f = tf.reshape(fnn, (curr_states.shape[0] , self.state_dim, 1))
        a = tf.matmul(two_times_x_times_P, f) ### LfV

        tildeg = self.g_nn(tildecurr_states)
        g = tf.stack([ D_y @ elem @ D_u for elem in tf.unstack(tildeg)])
        dVdx_times_g = tf.matmul(two_times_x_times_P , g)

        b =  tf.gather(dVdx_times_g, [1], axis = 2)
        a_b = tf.concat([a,b],axis = 2)
        ab = tf.reshape(a_b, (curr_states.shape[0], 2))
        sontags = tf.reshape(tf.map_fn(lambda elem : phi(elem[0],elem[1]) , ab ), (curr_states.shape[0],1,1))
       
        return sontags
    
     ###get the value of sontag law
    def dVsontag_value_CA0(self,curr_states):
        input_tensor = tf.reshape(curr_states, (curr_states.shape[0] , 1 , self.state_dim))
        #print('input_tensor',input_tensor)
        two_times_x_times_P = tf.stack([ 2 * elem @ tf.convert_to_tensor(P, dtype = tf.float32) for elem in tf.unstack(input_tensor)])
        #print('two_times_x_times_P',two_times_x_times_P)
        tildecurr_states = tf.stack([ elem @ D_x for elem in tf.unstack(input_tensor)])

        tildef_nn = tf.reshape(self.f_nn(tildecurr_states), (curr_states.shape[0] ,self.state_dim , 1))

        fnn = tf.stack([ D_y @ elem  for elem in tf.unstack(tildef_nn)])
    
        f = tf.reshape(fnn, (curr_states.shape[0] , self.state_dim, 1))
        a = tf.matmul(two_times_x_times_P, f) ### LfV

        tildeg = self.g_nn(tildecurr_states)
        g = tf.stack([ D_y @ elem @ D_u for elem in tf.unstack(tildeg)])
        dVdx_times_g = tf.matmul(two_times_x_times_P , g)

        b =  tf.gather(dVdx_times_g, [1], axis = 2)
       # print(b)
        a_b = tf.concat([a,b],axis = 2)
        ab = tf.reshape(a_b, (curr_states.shape[0], 2))
        sontags = tf.reshape(tf.map_fn(lambda elem : phi(elem[0],elem[1]) , ab ), (curr_states.shape[0],1,1))
        
        b2 = (2*1060*input_tensor[0,0,0] + (22+22)*input_tensor[0,0,1])*5
        b2 = tf.reshape(b2, (1 , 1 , 1))   ###get LgV1 for CA0
        a_b2 = tf.concat([a,b2],axis = 2)
        ab2 = tf.reshape(a_b2, (curr_states.shape[0], 2))
        sontags_CA0 = tf.reshape(tf.map_fn(lambda elem : phi_2(elem[0],elem[1]) , ab2 ), (curr_states.shape[0],1,1))
        return sontags_CA0
### this is dVdt for the NN model [\nabla V F_nn (x, sontag_nn(x))] but now remove states below \rho_s


    def dVsontag_modified(self,curr_states):
        input_tensor = tf.reshape(curr_states, (curr_states.shape[0] , 1 , self.state_dim))
        dVdt = tf.reshape(self.dVsontag(curr_states), (curr_states.shape[0],1,1))
        new_input = tf.concat([input_tensor, dVdt], axis = 2)
        inputs2 = tf.reshape(new_input , (curr_states.shape[0],3))
        x = tf.map_fn(lambda elem : modified_dVdt(elem[0],elem[1],elem[2]) , inputs2)
        return x

### this is dVdt for the FP model using NN sontang controller [\nabla V F (x, sontag_nn (x))] 

    def dVsontag_actual(self,curr_states):
        input_tensor = tf.reshape(curr_states, (curr_states.shape[0] , 1 , self.state_dim))
        two_times_x_times_P = tf.stack([ 2 * elem @ tf.convert_to_tensor(P, dtype = tf.float32) for elem in tf.unstack(input_tensor)])

        tildecurr_states = tf.stack([ elem @ D_x for elem in tf.unstack(input_tensor)])

        tildef_nn = tf.reshape(self.f_nn(tildecurr_states), (curr_states.shape[0] ,self.state_dim , 1))

        fnn = tf.stack([ D_y @ elem  for elem in tf.unstack(tildef_nn)])
        f = tf.reshape(fnn, (curr_states.shape[0] , self.state_dim, 1))
        a = tf.matmul(two_times_x_times_P, f) ### LfV
        tildeg = self.g_nn(tildecurr_states)
        g = tf.stack([ D_y @ elem @ D_u for elem in tf.unstack(tildeg)])
        dVdx_times_g = tf.matmul(two_times_x_times_P , g)
        b =  tf.gather(dVdx_times_g, [1], axis = 2)
        a_b = tf.concat([a,b],axis = 2)
        ab = tf.reshape(a_b, (curr_states.shape[0], 2))
        sontags = tf.reshape(tf.map_fn(lambda elem : phi(elem[0],elem[1]) , ab ), (curr_states.shape[0],1,1))
        concentrations = C_A0s + tf.zeros((curr_states.shape[0],1,1))
        inputs = tf.concat([input_tensor,concentrations,sontags], axis = 2 )
        actual_derivative = tf.map_fn( lambda x : CSTR_simulation_derivative2(x[0,0],x[0,1],x[0,2],x[0,3]), inputs )
        dVdt = tf.matmul(two_times_x_times_P,actual_derivative)
        return tf.reshape(dVdt, (curr_states.shape[0],1))

### this is dVdt for the FP model using NN sontang controller [\nabla V F (x, sontag_nn (x))] but now remove states below \rho_s

    def dVsontag_actual_modified(self,curr_states):
        input_tensor = tf.reshape(curr_states, (curr_states.shape[0] , 1 , self.state_dim))
        dVdt = tf.reshape(self.dVsontag_actual(curr_states), (curr_states.shape[0],1,1))
        new_input = tf.concat([input_tensor, dVdt], axis = 2)
        inputs2 = tf.reshape(new_input , (curr_states.shape[0],3))
        x = tf.map_fn(lambda elem : modified_dVdt(elem[0],elem[1],elem[2]) , inputs2)
        return x
    

        
def CSTR_simulation_derivative( C_A_initial, T_initial, C_A0, Q):
    
    C_A = C_A_initial + C_As
    T = T_initial + T_s
    
    dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
    dTdt = F / V * (T_0 - T) -  delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)
    
    return dCAdt, dTdt
   

import tensorflow as tf
from keras.losses import MeanSquaredError
from tensorflow.nn import relu


mse = tf.keras.losses.MeanSquaredError()

def custom_loss_function3(y_train, y_pred, model, X_batch, weight, threshold):
    X_batch_original = tf.convert_to_tensor(scaler_X.inverse_transform(X_batch), dtype = tf.float32)
    states = X_batch_original[:,0:2]
    dVdt = model.dVsontag_modified(states)
    reg_loss = tf.reduce_mean( relu(dVdt - threshold))
    loss1 =  mse(y_train,y_pred)
    loss2 =  reg_loss 
    return mse(y_train,y_pred) + weight * reg_loss, loss1, loss2





In [16]:
## training process

threshold = -10
weight = 0.001
num_epochs = 1
batchsize = 250
mse = tf.keras.losses.MeanSquaredError()
### this creates the model
model = ControlAffineNN(2,2,(10,10),(10,))

##
opt = tf.keras.optimizers.Adam(learning_rate=0.0007)
for epoch in range(num_epochs):
  for step in range(X_train.shape[0]//batchsize):
    start_idx = batchsize*step
    end_idx = batchsize*(step+1)
    X_batch = X_train[start_idx:end_idx]
    y_batch = y_train[start_idx:end_idx]
    with tf.GradientTape() as tape:
      pred = model(X_batch)
      loss , loss1 , loss2 = custom_loss_function3(y_batch, pred, model, X_batch, weight, threshold) 
    grads = tape.gradient(loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))
  print("Epoch : " , epoch, " Loss: " , loss, "Loss1", loss1.numpy(), "Loss2", loss2.numpy())

### save weights




250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
250
Epoch :  0  Loss:  tf.Tensor(1.1252172, shape=(), dtype=float32) Loss1 1.1252002 Loss2 0.017098267


In [17]:
model.load_weights("test_new_region3") ###0.0005 for 200 epoch, then 

print(mse(model(X_train),y_train)) ###the training error
pred = model(X_test)
print(mse(pred,y_test))  ###the testing error

8750
tf.Tensor(5.337881884770468e-05, shape=(), dtype=float64)
3750
tf.Tensor(5.510313349077478e-05, shape=(), dtype=float64)


# Test the closed-loop performance

In [54]:
#sontag law using LSNN
NN_input_tf = tf.convert_to_tensor(NN_input, dtype = tf.float32) ### convert the array to tensor

print('NN_input_tf', NN_input_tf)  ###get the input in real value

states = NN_input_tf[:,0:2]
print('states',states)
dVdt = model.dVsontag(states) ###using state to calculate the dVdt for the NN model using LSNN, 
print('dVdt', dVdt)

k = 0
for i in range(50412):   ###tell the number of initial states which dvdt is positive
    if dVdt[i] <0:
        k = k+1
print(k)

NN_input_tf tf.Tensor(
[[ 1.5537595e+00 -7.6436089e+01 -3.5000000e+00 -5.0000000e+05]
 [ 1.5687970e+00 -7.6436089e+01 -3.5000000e+00 -5.0000000e+05]
 [ 1.5838345e+00 -7.6436089e+01 -3.5000000e+00 -5.0000000e+05]
 ...
 [-1.5590225e+00  7.6195488e+01  3.5000000e+00  5.0000000e+05]
 [-1.5439850e+00  7.6195488e+01  3.5000000e+00  5.0000000e+05]
 [-1.5289474e+00  7.6195488e+01  3.5000000e+00  5.0000000e+05]], shape=(50412, 4), dtype=float32)
states tf.Tensor(
[[  1.5537595 -76.43609  ]
 [  1.568797  -76.43609  ]
 [  1.5838345 -76.43609  ]
 ...
 [ -1.5590225  76.19549  ]
 [ -1.543985   76.19549  ]
 [ -1.5289474  76.19549  ]], shape=(50412, 2), dtype=float32)
dVdt tf.Tensor(
[[ -871.69324]
 [ -860.7048 ]
 [ -844.938  ]
 ...
 [-6264.846  ]
 [-6081.0635 ]
 [-5901.076  ]], shape=(50412, 1), dtype=float32)
50312


In [55]:
#sontag law using LSNN
NN_input_tf = tf.convert_to_tensor(NN_input, dtype = tf.float32) ### convert the array to tensor

print('NN_input_tf', NN_input_tf)  ###get the input in real value

states = NN_input_tf[:,0:2]
print('states',states)
dVdt = model.dVsontag_actual(states) ###using state to calculate the dVdt for the FP model using LSNN, 
print('dVdt', dVdt)

k = 0
for i in range(50412):   ###tell the number of initial states which dvdt is positive
    if dVdt[i] <0:
        k = k+1
    else:
        print(dVdt[i])
print(k)

NN_input_tf tf.Tensor(
[[ 1.5537595e+00 -7.6436089e+01 -3.5000000e+00 -5.0000000e+05]
 [ 1.5687970e+00 -7.6436089e+01 -3.5000000e+00 -5.0000000e+05]
 [ 1.5838345e+00 -7.6436089e+01 -3.5000000e+00 -5.0000000e+05]
 ...
 [-1.5590225e+00  7.6195488e+01  3.5000000e+00  5.0000000e+05]
 [-1.5439850e+00  7.6195488e+01  3.5000000e+00  5.0000000e+05]
 [-1.5289474e+00  7.6195488e+01  3.5000000e+00  5.0000000e+05]], shape=(50412, 4), dtype=float32)
states tf.Tensor(
[[  1.5537595 -76.43609  ]
 [  1.568797  -76.43609  ]
 [  1.5838345 -76.43609  ]
 ...
 [ -1.5590225  76.19549  ]
 [ -1.543985   76.19549  ]
 [ -1.5289474  76.19549  ]], shape=(50412, 2), dtype=float32)
dVdt tf.Tensor(
[[ -969.6623]
 [ -953.4345]
 [ -932.4526]
 ...
 [-6474.349 ]
 [-6320.536 ]
 [-6164.3687]], shape=(50412, 1), dtype=float32)
tf.Tensor([229.6527], shape=(1,), dtype=float32)
tf.Tensor([167.82755], shape=(1,), dtype=float32)
tf.Tensor([165.73782], shape=(1,), dtype=float32)
tf.Tensor([3.7533264], shape=(1,), dtype=float32)


tf.Tensor([229.6527], shape=(1,), dtype=float32)
tf.Tensor([167.82755], shape=(1,), dtype=float32)
tf.Tensor([165.73782], shape=(1,), dtype=float32)
tf.Tensor([3.7533264], shape=(1,), dtype=float32)
tf.Tensor([86.966034], shape=(1,), dtype=float32)
tf.Tensor([9.788116], shape=(1,), dtype=float32)
tf.Tensor([342.0528], shape=(1,), dtype=float32)
tf.Tensor([15.365921], shape=(1,), dtype=float32)
tf.Tensor([74.41705], shape=(1,), dtype=float32)
tf.Tensor([9.972595], shape=(1,), dtype=float32)
tf.Tensor([1.0393066], shape=(1,), dtype=float32)
tf.Tensor([238.24918], shape=(1,), dtype=float32)
tf.Tensor([229.51483], shape=(1,), dtype=float32)
tf.Tensor([119.134995], shape=(1,), dtype=float32)
tf.Tensor([368.42004], shape=(1,), dtype=float32)
tf.Tensor([95.15317], shape=(1,), dtype=float32)
tf.Tensor([231.51126], shape=(1,), dtype=float32)
tf.Tensor([218.089], shape=(1,), dtype=float32)
tf.Tensor([13.993591], shape=(1,), dtype=float32)
tf.Tensor([84.02782], shape=(1,), dtype=float32)
tf.Tenso

tf.Tensor([139.15039], shape=(1,), dtype=float32)
tf.Tensor([11.376953], shape=(1,), dtype=float32)
tf.Tensor([1.4893188], shape=(1,), dtype=float32)
tf.Tensor([2.5638428], shape=(1,), dtype=float32)
tf.Tensor([2.4250793], shape=(1,), dtype=float32)
tf.Tensor([18.165405], shape=(1,), dtype=float32)
tf.Tensor([229.6527], shape=(1,), dtype=float32)
tf.Tensor([167.82755], shape=(1,), dtype=float32)
tf.Tensor([165.73782], shape=(1,), dtype=float32)
tf.Tensor([3.7533264], shape=(1,), dtype=float32)
tf.Tensor([86.966034], shape=(1,), dtype=float32)
tf.Tensor([9.788116], shape=(1,), dtype=float32)
tf.Tensor([342.0528], shape=(1,), dtype=float32)
tf.Tensor([15.365921], shape=(1,), dtype=float32)
tf.Tensor([74.41705], shape=(1,), dtype=float32)
tf.Tensor([9.972595], shape=(1,), dtype=float32)
tf.Tensor([1.0393066], shape=(1,), dtype=float32)
tf.Tensor([238.24918], shape=(1,), dtype=float32)
tf.Tensor([229.51483], shape=(1,), dtype=float32)
tf.Tensor([119.134995], shape=(1,), dtype=float32)
tf.T

tf.Tensor([139.15039], shape=(1,), dtype=float32)
tf.Tensor([11.376953], shape=(1,), dtype=float32)
tf.Tensor([1.4893188], shape=(1,), dtype=float32)
tf.Tensor([2.5638428], shape=(1,), dtype=float32)
tf.Tensor([2.4250793], shape=(1,), dtype=float32)
tf.Tensor([18.165405], shape=(1,), dtype=float32)
tf.Tensor([229.6527], shape=(1,), dtype=float32)
tf.Tensor([167.82755], shape=(1,), dtype=float32)
tf.Tensor([165.73782], shape=(1,), dtype=float32)
tf.Tensor([3.7533264], shape=(1,), dtype=float32)
tf.Tensor([86.966034], shape=(1,), dtype=float32)
tf.Tensor([9.788116], shape=(1,), dtype=float32)
tf.Tensor([342.0528], shape=(1,), dtype=float32)
tf.Tensor([15.365921], shape=(1,), dtype=float32)
tf.Tensor([74.41705], shape=(1,), dtype=float32)
tf.Tensor([9.972595], shape=(1,), dtype=float32)
tf.Tensor([1.0393066], shape=(1,), dtype=float32)
tf.Tensor([238.24918], shape=(1,), dtype=float32)
tf.Tensor([229.51483], shape=(1,), dtype=float32)
tf.Tensor([119.134995], shape=(1,), dtype=float32)
tf.T

tf.Tensor([139.15039], shape=(1,), dtype=float32)
tf.Tensor([11.376953], shape=(1,), dtype=float32)
tf.Tensor([1.4893188], shape=(1,), dtype=float32)
tf.Tensor([2.5638428], shape=(1,), dtype=float32)
tf.Tensor([2.4250793], shape=(1,), dtype=float32)
tf.Tensor([18.165405], shape=(1,), dtype=float32)
49700


In [56]:
print(states)

tf.Tensor(
[[  1.5537595 -76.43609  ]
 [  1.568797  -76.43609  ]
 [  1.5838345 -76.43609  ]
 ...
 [ -1.5590225  76.19549  ]
 [ -1.543985   76.19549  ]
 [ -1.5289474  76.19549  ]], shape=(50412, 2), dtype=float32)
